## Description

##### Use to develop any generic DNN based on input of layers and their nr of neurons

## Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from keras.utils.np_utils import to_categorical

## Experiment with Tensorflow Basics

In [ ]:
array = np.array([[1, 2, 4], [4, 5, 6]])

In [ ]:
tf_array = tf.Variable(array).shape

In [ ]:
tf.reduce_mean(array, 0)

## Basic Neural Network Class

### Weight and bias experimentation

In [ ]:
x = tf.constant(np.array([1, 2, 3, 4]), dtype=tf.float32)
y = tf.constant(np.array([2, 4, 6, 8]), dtype=tf.float32)

In [ ]:
x

In [ ]:
# initialize weights and bias with random tensor values
w = tf.Variable(np.random.randn(), dtype="float32")
b = tf.Variable(np.random.randn(), dtype="float32")

In [ ]:
n_layers = [4, 3, 3, 1]

In [ ]:
ran1 = tf.random.normal([n_layers[0], n_layers[1]], stddev=0.1)

In [ ]:
ran1

In [ ]:
w1 = tf.Variable(ran1, name='W1')

In [ ]:
b1 = tf.zeros([1, n_layers[2]])

In [ ]:
x_in = tf.Variable(tf.random.normal([1, 4]))

In [ ]:
Z1 = tf.matmul(x_in, w1)

In [ ]:
Z1

### Data Setup

In [ ]:
x = tf.constant(np.array([[1, 2, 3, 4],
                         [1, 3, 5, 6],
                         [2, 5, 6, 7],
                         [3, 5, 7, 8]
                         ]), dtype=tf.float32)

In [ ]:
data = np.array([1, 2, 3, 3])
shape = (data.size, data.max()+1)
one_hot = np.zeros(shape)
rows = np.arange(data.size)
y = one_hot[rows, data] = 1
y = one_hot

### NN Class

In [ ]:
class GenericNeuralNetwork:
    
    """
    This class builds any neural network for any given number of layers and their neuron units
    """
    
    def __init__(self, n_layers: list):
        """
        constructor
        :param n_layers: list of layers of neural network from input to output containing number of nodes/units
        in each layer of the network
        """
        
        # store the parametrs of network
        self.weights = []
        self.biases = []
        self.params = [] # store alternative weights and biases
        
        # declare layer-wise weights and biases. NOTE: each layer weight matrix in layer j 
        # = num_in_layer_j-1 * num_in_layer_j
        for i in range(0, len(n_layers)-1, 1):
            w_x = tf.Variable(tf.random.normal([n_layers[i], n_layers[i+1]], stddev=0.1), name=f'W{i+1}')
            b_x = tf.Variable(tf.zeros([1, n_layers[i+1]]), name=f'B{i+1}')
            self.weights.append(w_x)
            self.biases.append(b_x)
        self.params = [w for b in zip(self.weights, self.biases) for w in b]
         

    def forward(self, x):
        """
        Forward pass of the neural network
        :pararm x: input data
        :return: predicted logits
        """
        Z_list = [] # store all layer's activation outputs

        # calculate first layer output from inputs
        X_tf = tf.cast(x, dtype=tf.float32)
        Z1 = tf.matmul(X_tf, self.weights[0]) + self.biases[0]
        Z1 = tf.nn.relu(Z1)

        Z_list.append(Z1)

        # for remaining layers compute all activation outputs
        j = 0 # counter for Z_list
        for i in range(1, len(self.weights), 1): # first layer already computed so start at index 1
            Z_i = tf.matmul(Z_list[j], self.weights[i]) + self.biases[i]
            if i != len(self.weights) - 1: # NOTE: for last layer, we do not apply an activation (for logits)
                Z_i = tf.nn.relu(Z_i)  
            Z_list.append(Z_i)
            j += 1

        return Z_list[-1] # logit layer (final layer before activation)
    
    
    def loss(self, y_true, logits):
        """
        logits = Tensor of shape (batch_size, size_output)
        y_true = Tensor of shape (batch_size, size_output)
        """
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, 1)), dtype=tf.float32)
        logits_tf = tf.cast(tf.reshape(logits, (-1, 1)), dtype=tf.float32)
        return tf.compat.v1.losses.sigmoid_cross_entropy(y_true_tf, logits_tf)
    
    
    def backprop(self, x, y):
        """
        Back propagation algorithm to calculate weights and biases
        :param x: input data
        :param y: true labeled data
        """
        optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
        with tf.GradientTape() as tape:
            predicted = self.forward(x)
            current_loss = self.loss(y_true=y, logits=predicted)
        grads = tape.gradient(current_loss, self.params)
        optimizer.apply_gradients(zip(grads, self.params),
                                 global_step=tf.compat.v1.train.get_or_create_global_step())
        print(f"Current loss {tf.reduce_mean(current_loss)}")

### Run Algorithm

In [ ]:
# initialize DNN class
generic_neural_network = GenericNeuralNetwork(n_layers=[4, 3, 4])

In [ ]:
logit_layer_output = generic_neural_network.forward(x=x)
logit_layer_output_relu = tf.nn.sigmoid(logit_layer_output)
logit_layer_output

In [ ]:
num_epochs = 100
for e in range(num_epochs):
    generic_neural_network.backprop(x=x, y=y)

In [ ]:
generic_neural_network.weights

In [ ]:
predict = generic_neural_network.forward(x=x)

In [ ]:
predict

In [ ]:
predict = tf.nn.sigmoid(predict)

In [ ]:
predict